# Elasticsearch Queries


## Setup


In [1]:
from elasticsearch import Elasticsearch
from connectors.elk import Elk
import sys
import json
sys.path.append('..')

elk = Elk()
es: Elasticsearch = elk.es

Elasticsearch: Connection sucessful


## Users not connected from more than X days


In [2]:
from datetime import datetime, timedelta

# Definición de la consulta con top_hits para obtener detalles adicionales
query = {
    "size": 0,
    "aggs": {
        "users": {
            "terms": {
                "field": "userId.keyword",
                "size": 10000  # Ajusta este número según la cantidad esperada de usuarios
            },
            "aggs": {
                "latest_login": {
                    "max": {
                        "field": "@timestamp"
                    }
                },
                "top_user_info": {
                    "top_hits": {
                        "sort": [
                            {
                                "@timestamp": {
                                    "order": "desc"
                                }
                            }
                        ],
                        "_source": {
                            "includes": ["userDisplayName", "deviceDetail.displayName"]
                        },
                        "size": 1
                    }
                }
            }
        }
    }
}

# Realiza la consulta
# Asegúrate de cambiarlo por el nombre real de tu índice
index_name = "logs-ms_singins"
result = es.search(index=index_name, body=query)

# Extrae y ordena los resultados por la fecha de última conexión de manera ascendente
users_last_login = []
for bucket in result['aggregations']['users']['buckets']:
    user_id = bucket['key']
    last_login_timestamp = bucket['latest_login']['value_as_string']
    user_info = bucket['top_user_info']['hits']['hits'][0]['_source']
    user_display_name = user_info.get('userDisplayName', 'N/A')
    device_display_name = user_info.get(
        'deviceDetail', {}).get('displayName', 'N/A')
    users_last_login.append(
        (user_id, user_display_name, device_display_name, last_login_timestamp))

# Ordena por la fecha de última conexión de manera ascendente
users_last_login_sorted = sorted(users_last_login, key=lambda x: x[3])

# Define el formato de fecha utilizado en tus timestamps
fecha_formato = "%Y-%m-%dT%H:%M:%S.%fZ"

# Calcula la fecha límite para los usuarios inactivos (hace más de 30 días)
limite_inactividad = datetime.now() - timedelta(days=30)

# Filtra los usuarios inactivos por más de 30 días
usuarios_inactivos = []
for user in users_last_login_sorted:
    user_id, user_display_name, device_display_name, last_login_str = user
    last_login = datetime.strptime(last_login_str, fecha_formato)

    if last_login < limite_inactividad:
        usuarios_inactivos.append(user)

# Imprime los usuarios inactivos
for user in usuarios_inactivos:
    print(user)

('8f20ce5e-0da7-432d-a91f-c04614c406c4', 'Santana, Fernando', '', '2024-01-24T20:34:50.000Z')
('77d6c11b-92a4-45c6-be2e-e6043f11ff00', 'Cabral, Ivonne', '', '2024-01-25T14:02:57.000Z')
('90042398-f05b-4405-8690-f695079ff3e4', 'tdg.galera1', '', '2024-01-25T14:22:30.000Z')
('517439f8-c726-4f42-a251-3f7de24b1015', 'Gutierres Tovar, Victor', '', '2024-01-26T09:34:00.000Z')
('78678837-7233-4487-8646-891964de3b68', 'Tabacalera Reports', '', '2024-01-29T07:56:43.000Z')


## User searcher


In [30]:
response = es.search(
    index="ms_users",
    query={
        "multi_match": {
            "query":    "victor gutierrez",
            "fields": ["displayName", "deviceDisplayName"]
        }
    }
)

print(json.dumps(dict(response), indent=2))
print(json.dumps(response["hits"]["hits"][0]["_source"]["mail"], indent=2))

{
  "took": 1,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 7,
      "relation": "eq"
    },
    "max_score": 10.7018385,
    "hits": [
      {
        "_index": "ms_users",
        "_id": "49d17a6e-0730-4fb2-8eb6-aa78dfe0cb00",
        "_score": 10.7018385,
        "_source": {
          "assignedLicenses": [
            {
              "disabledPlans": [],
              "skuId": "26124093-3d78-432b-b5dc-48bf992543d5"
            },
            {
              "disabledPlans": [],
              "skuId": "05e9a617-0261-4cee-bb44-138d3ef5d965"
            }
          ],
          "businessPhones": [],
          "preferredLanguage": null,
          "mail": "victor.gutierrez@tabacaleracigar.com",
          "mobilePhone": null,
          "officeLocation": "Madrid",
          "displayName": "Gutierrez, Victor",
          "surname": "Gutierrez",
          "givenName": "Victor",
     

## Last connection from a user


In [12]:
# Define el userId para el cual quieres obtener la última conexión
user_id = "21bf1acf-0ebe-420e-b385-9d4014a11406"

# Construye la consulta
query = {
    "query": {
        "match": {
            "userId": user_id
        }
    },
    "sort": [
        {
            "@timestamp": {
                "order": "desc"
            }
        }
    ],
    "size": 1
}

# Realiza la consulta al índice deseado
index_name = "logs-ms_singins"
response = es.search(index=index_name, body=query)


print(json.dumps(dict(response), indent=4))

# Extrae y muestra la información relevante de la respuesta
if response["hits"]["hits"]:
    last_login = response["hits"]["hits"][0]["_source"]
    print(f"Última conexión de {last_login['userDisplayName']} ({last_login['userPrincipalName']}):")
    print(f"Fecha y hora: {last_login['@timestamp']}")
    print(f"IP: {last_login['ipAddress']}")
    print(f"Dispositivo: {last_login['deviceDetail']['displayName']}, {last_login['deviceDetail']['operatingSystem']}")
else:
    print("No se encontraron conexiones para el usuario especificado.")

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 206,
            "relation": "eq"
        },
        "max_score": null,
        "hits": [
            {
                "_index": "logs-ms_singins",
                "_id": "4878c2df-c35a-4b1c-abc9-30b949c72c00",
                "_score": null,
                "_source": {
                    "createdDateTime": "2024-02-29T09:16:37Z",
                    "userDisplayName": "Hidalgo Casta\u00f1o, Gema Belen",
                    "userPrincipalName": "gema.hidalgo@tabacaleracigar.com",
                    "userId": "21bf1acf-0ebe-420e-b385-9d4014a11406",
                    "appId": "38aa3b87-a06d-4817-b275-7a316988d93b",
                    "appDisplayName": "Windows Sign In",
                    "ipAddress": "195.53.89.98",
                    "clientAppUsed": "Mobile Apps and Desktop

# Reindex

## change a field type

In [17]:
# Define el nombre del nuevo índice
index_name = "ms_users2"

# # Define el mapeo
# mapping = {
#     "mappings": {
#         "properties": {
#             "id": {"type": "keyword"},
#             "last_signin": {"type": "date"}
#         }
#     }
# }

# # Crea el índice con el mapeo definido
# if not es.indices.exists(index=index_name):
#     es.indices.create(index=index_name, body=mapping)
#     print(f"Índice {index_name} creado con éxito.")
# else:
#     print(f"El índice {index_name} ya existe.")

# Define el nombre del índice antiguo
old_index_name = "ms_users"

# Reindexa los datos del índice antiguo al nuevo índice
reindex_body = {
    "source": {"index": old_index_name},
    "dest": {"index": index_name}
}
es.reindex(body=reindex_body)

print(f"Datos reindexados de {old_index_name} a {index_name}.")


Datos reindexados de ms_users a ms_users2.
